In [ ]:
#!/usr/bin/env python
# coding: utf-8

Using Your Own Connectivity<br>
=====================================<br>
<br>
This tutorial covers a basic mechanism for using your own connectivity data <br>
in TVB's scripting interface. It describes a simple text file based format<br>
and how to load data from that format.

A Simple Text File Based Format<br>
-------------------------------<br>
<br>
In this tutorial we will use a "simple" text file based data format that is<br>
supported by TVB. It consists of text files (optionally compressed) containing<br>
the the main data elements that make up a Connectivity, eg. weights, <br>
tract-lengths, etc. This format is very simple but not particularly safe, in <br>
the sense that the lack of explicit meta-information means detailed consistency<br>
checks of loaded data aren't really possible, so **be careful**...<br>
<br>
The format is just a directory containing the following files (the contents, which<br>
are described in detail below):<br>
    <br>
    info.txt<br>
    weights.txt<br>
    tract_lengths.txt<br>
    cortical.txt<br>
    centres.txt<br>
    average_orientations.txt<br>
    areas.txt<br>
<br>
We can use bzip2 to compress each of these files, with the exception of the <br>
info.txt file, and everything will work the same. This is useful for saving <br>
storage space when we want to work with either very many or very large <br>
connectivity datasets.<br>
<br>
A more detailed description of each file follows. A number of sample connectivity<br>
data sets can be found in tvb_data.connectivity.<br>
<br>
**info.txt**<br>
    This file contains a summary about the connectivity matrix (eg, distance units, weights units), a vector to distinguish left <br>
    from right areas, etc.<br>
<br>
**weights.txt**<br>
    This file contains the connectome, that is a (un)weighted matrix whose values may represent units such as fibre density. <br>
    Zero entries in thsi matrix mean 'unconnected' areas.<br>
<br>
**tract_lengths.txt**<br>
    This file contains . <br>
    **NOTE:** *If no tract-length data is provided, TVB can calculate tract-lengths <br>
    based on the Euclidean distance between centres.*<br>
<br>
**cortical.txt**<br>
    This file contains a vector providing a means of distinguishing cortical from subcortical regions.<br>
<br>
**centres.txt**<br>
    This file contains the region labels (first columnn) and the list of Cartesian triplets (x,y,z) that specify <br>
    the spatial location rof each region.<br>
<br>
**average_orientations.txt**<br>
    This file contains the average 'normal vectors' of the dipoles (regions). They are derived from the cortical surface normal  <br>
    vectors and used to compute the forward solutions of EEG and MEG. <br>
<br>
**areas.txt**<br>
    This file contains a vector of length number of regions specifying the area in [mm2] that each region represents. <br>
    <br>


Where To Put Your Data<br>
----------------------<br>
<br>
If you want to use relative paths to your data in scripts, then the data needs to be<br>
in the tvb_data directory. So putting your data there can simplify things<br>
and make your scripts more portable.<br>
<br>
However, you can place your data any where you want, it just requires that you specify<br>
absolute path names in your scripts.

Setup<br>
-----<br>
<br>
We'll first import some stuff we'll need via the "lab", this has the side<br>
effect of turning off TVB's storage for us, which is, for the moment, <br>
necessary when using the scripting interface to TVB.

In[1]:

In [ ]:
get_ipython().magic('pylab nbagg')

In[2]:

In [ ]:
from tvb.simulator.lab import *
LOG = get_logger('demo')
from tvb.simulator.plot.tools import *

Loading Your Data<br>
-----------------<br>
<br>
To load your data we just need to specify its path <br>
when we create a Connectivity object. We'll use a very simple ("toy") connectivity dataset<br>
called paupau, which is included in TVB, as our first example. This connectivity dataset <br>
consists of just 4 regions.<br>
<br>
All we need to do is simply create a Connectivity object specifying the path to our data<br>
in the call. This is just a string that can be either a relative path (w.r.t. <br>
tvb_data), or an absolute path to our data.

In[3]:

oad the connectivity data in the directory tvb_data.connectivity.paupau

In [ ]:
conn_pau = connectivity.Connectivity.from_file("paupau.zip")

Where the use of os.path.join is for portability between operating systems, for example, it resolves<br>
to "connectivity/paupau" on Linux and "connectivity\paupau" on windows.

We can now configure the Connectivity object we've just created and take a quick look<br>
at the properties of our data using:

In[4]:

onfigure, to compute derived data, such as number_of_nodes and delays

In [ ]:
conn_pau.configure()
conn_pau.summary_info

And we can look at simple matrix view of weights and tract-lengths data using plot_connectivity

In[5]:

In [ ]:
plot_connectivity(connectivity = conn_pau)

Multiple datasets in one<br>
-------------------------<br>
<br>
It is possible to reuse the majority of a dataset and have multiple pieces of <br>
data for one or more elements. That is, by default, when you create a Connectivity<br>
object by pointing at a specific data directory, the data components are expected<br>
to be in files with specific names, such as weights.txt.bz2. However, it is possible<br>
for a specific connectivity data directory to contain multiple weights files as long<br>
as they are consistent with the rest of the connectivity data. For example, in <br>
addition weights.txt.bz2 our directory may contain files named:<br>
<br>
    weights_0.txt.bz2<br>
    weights_bob.txt.bz2<br>
    hfdbvaerut93483489.txt.bz2<br>
<br>
thought that last one is a bad name as it's not clear what it contains.<br>
<br>
The loading the connectivity specifying a particular weights would become:<br>
<br>
    pau_path = os.path.join("connectivity", "paupau")<br>
    conn_pau = connectivity.Connectivity(default = pau_path, weights_file = "weights_bob.txt.bz2")

A Larger Example Dataset<br>
-------------------------<br>
<br>
Load some more complicated data... to emphasise visualisation techniques

In[6]:

oad the connectivity data in the directory tvb_data.connectivity.connectivity_190

In [ ]:
conn_190 = connectivity.Connectivity.from_file("connectivity_192.zip")
#conn_190.default.reload(conn_190, os.path.join("connectivity", "o52r00_irp2008_hemisphere_both_subcortical_true_regions_190"))
#Configure, to compute derived data, such as number_of_nodes and delays
conn_190.configure()
conn_190.summary_info

Again, we can just take a look at a matrix plot of weights and tract lengths

In[7]:

In [ ]:
plot_connectivity(connectivity = conn_190)

Scaling, Transforming and Resampling<br>
-------------------------------------<br>
<br>
The Connectivity object includes a number of methods for scaling, transforming or resampling a connectivity.<br>
<br>
**Scaling**<br>
___________<br>
<br>
Two simple types of scaling are supported and the ```scaling mode``` is the following: <br>
            <br>
* **tract**: Scale by a value such that the maximum absolute value of a single connection is 1.<br>
            <br>
* **region**: Scale by a value such that the maximum absolute value of the cumulative input to any region is 1.

In[8]:

nitialise a Connectivity object

In [ ]:
orig_wm = connectivity.Connectivity(load_default=True)
orig_wm.configure()

In [ ]:
scaled_wm = connectivity.Connectivity(load_default=True)
scaled_wm.configure()

cale weights by the maximum aboslute value

In [ ]:
scaled_wm.weights = orig_wm.scaled_weights(mode='tract')
plot_connectivity(scaled_wm, num="tract_mode", plot_tracts=False)

NOTE: Currently, multiple **tract** and/or **region** scalings without intermediate **none** scaling mode destroy the ability to recover the original un-scaled weights matrix.

In[9]:

In [ ]:
scaled_wm.weights = orig_wm.scaled_weights(mode='none')
scaled_wm.weights = orig_wm.scaled_weights(mode='region')
plot_connectivity(scaled_wm, num="region_mode", plot_tracts=False)

In[10]:

ndo scaling

In [ ]:
scaled_wm.weights = orig_wm.scaled_weights(mode='none')
plot_connectivity(scaled_wm, num="default_mode", plot_tracts=False)

**Transforming**<br>
_________________<br>
<br>
These methods will affect either the topology or the distribution of the weights matrix. <br>
<br>
* **binarize matrix**: Sets all the non-zero values to 1. In other words it tyields the adjacency matrix.<br>
* ** remove self-connections ** : Sets to zero all the elements of the main diagonal matrix.<br>
<br>
NOTE: these methods do not provide a means to undo the changes<br>


In[11]:

inarize

In [ ]:
scaled_wm.weights = orig_wm.transform_binarize_matrix()
plot_matrix(scaled_wm.weights, connectivity=scaled_wm, binary_matrix=True)